In [2]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from tqdm import tqdm
import cv2
from PIL import Image
from skimage import exposure
from skimage.transform import resize
import scipy
import time
from statistics import mean
import tensorflow as tf
import keras as ke
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow.keras.layers import Input
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
Tabela_treino = pd.read_csv('/kaggle/input/spr-x-ray-age/train_age.csv',sep = ',',engine = 'python')
display(Tabela_treino)

,imageId,age
0,0,89.0
1,1,72.0
2,2,25.0
3,3,68.0
4,4,37.0
...,...,...
10697,10697,79.0
10698,10698,67.0
10699,10699,66.0
10700,10700,78.0


In [6]:
nlin = np.shape(Tabela_treino)[0]   #numero de linhas da minha tabela do excel
hhh = []
tamy = []

for i in tqdm(range(0,nlin)):
    imageID = Tabela_treino['imageId'].iloc[i]      #pega o valor do imageID de cada linha 
    file_path = ("/kaggle/input/spr-x-ray-age/kaggle/kaggle/train/"  + str(imageID).zfill(6) + ".png")
    img = cv2.imread(file_path) #aplica o numero do imageID pra buscar a imagem no diretório com o mesmo numero
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.merge((img,img,img))
    img = cv2.resize(img, (200,200), interpolation = cv2.INTER_AREA)
    img = np.array(img)/255
    hhh.append(img)
    tamy.append(Tabela_treino['age'].iloc[i])
tamy = np.array(tamy)
    

100%|██████████| 10702/10702 [09:21<00:00, 19.07it/s]


In [9]:
X_treino, X_teste, Y_treino, Y_teste = train_test_split(hhh, tamy, test_size=0.2, random_state=50,shuffle=True)

In [7]:
tamy = 0 #list(Y_all).clear()
hhh.clear()

print(tamy, hhh)
print(np.shape(X_treino), np.shape(X_teste), np.shape(Y_treino), np.shape(Y_teste))

0 []
(8561, 200, 200, 3) (2141, 200, 200, 3) (8561,) (2141,)


In [8]:
start_time = time.time()
model_extract = tf.keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

features_treino = model_extract.predict(np.array(X_treino))
features_teste = model_extract.predict(np.array(X_teste))
print("--- %s seconds ---" % (time.time() - start_time))

67/67 [==============================] - 579s 9s/step
--- 2925.0612967014313 seconds ---


In [9]:
X_treino.clear()
X_teste.clear()

In [13]:
print(np.shape(features_treino))
print(np.shape(features_teste))

(8561, 6, 6, 512)
(2141, 6, 6, 512)


In [ ]:
from keras.applications import EfficientNetB2
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

start_time = time.time()

#iniciando as listas
mae_por_fold = []
loss_por_fold = []
all_models=[]
all_loss=[]
all_mae=[]
all_vloss=[]
all_vmae=[]

#Dados de entrada
inputs = np.array(X_treino)
targets = Y_treino

# escolher o numero de folds
num_folds = 5

# Validação cruzada
kfold = KFold(n_splits=num_folds, shuffle=True)
fold_no = 1
for train, test in kfold.split(inputs, targets):

    # criando a base da ResNet50 pré-treinada
    base_model = EfficientNetB2(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

    # adicionando camadas de classificação no topo da base_model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(240, activation='relu')(x)
    predictions = Dense(1)(x)

    # definindo o modelo final
    model = Model(inputs=base_model.input, outputs=predictions)

    # congelando as camadas da base_model
    for layer in base_model.layers:
        layer.trainable = False

    opt = ke.optimizers.Adam( learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False)
    #callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
    model.compile(optimizer= opt, loss='mse',metrics=['mae'])  # Compilando o modelo
    

    print('------------------------------------------------------------------------')
    print(f'Treinamento para o fold {fold_no} ...')
    
    # Fit data to model
    history = model.fit(inputs[train], targets[train],
              batch_size=10,
              #callbacks=[callback],
              epochs=100,
              validation_data=(inputs[test], targets[test]))

  
    mse, mae =  model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score para o fold {fold_no}: {model.metrics_names[0]} de {mse}; {model.metrics_names[1]} de {mae}')
    mae_por_fold.append(mae) # aqui estamos criando uma lista com as accs dos 5 folds
    loss_por_fold.append(mse)    # aqui estamos criando uma lista com as loss dos 5 folds
    
   # Increase fold number
    fold_no = fold_no + 1
    
    
    all_models.append(model)
    
    all_vloss.append(history.history["val_loss"])
    all_vmae.append(history.history["val_mae"])
    all_loss.append(history.history["loss"])
    all_mae.append(history.history["mae"])
    
print("--- %s seconds ---" % (time.time() - start_time))

31790344/31790344 [==============================] - 2s 0us/step
------------------------------------------------------------------------
Treinamento para o fold 1 ...
